In [40]:
import pandas as pd
import requests
import os
import json
import time
import base64
from dotenv import load_dotenv


## Leemos los datos que nos ha dado Ciber desde la web de AWS para no tener que descargar en el ordenador

In [33]:
# Link en el AWS: Propiedades > URL del objeto
#url1 = "https://desafiogrupo1.s3.us-east-1.amazonaws.com/phishing.json"
#df1 = pd.read_json(url1)
df1.info

<bound method DataFrame.info of                                          _id  \
0       {'$oid': '662ef11c9c88568844e38f1a'}   
1       {'$oid': '660caa1a528344e2948e0c13'}   
2       {'$oid': '65c4d948ec20407e161504ca'}   
3       {'$oid': '65d75308bdfee21c31b781fa'}   
4       {'$oid': '660f4d1a528344e29492a240'}   
...                                      ...   
164420  {'$oid': '65ea7129528344e2945b0ef2'}   
164421  {'$oid': '6521d6e2b538b084973653df'}   
164422  {'$oid': '6409c08a832fbf5d0e34576b'}   
164423  {'$oid': '6409c087832fbf5d0e340565'}   
164424  {'$oid': '6521d6e2b538b08497364996'}   

                                              domain_name  category  \
0                                       conflrma57843.com  phishing   
1                                         www.zgahdjm.com  phishing   
2                                          attt.pages.dev  phishing   
3                                 chermate.weeblysite.com  phishing   
4                             allegr

Hacemos una lista de urls con la intención de pasarlas por VT y que nos de los datos que ciber necesita para el enriquecimiento

In [47]:
urls = df1['url'][0:2]

listapd = pd.DataFrame(urls).reset_index(drop=True)

listapd.info()

listapd.columns = ['url'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     2 non-null      object
dtypes: object(1)
memory usage: 148.0+ bytes


## Llamamos a VT

In [48]:
load_dotenv()

API_KEY = os.environ.get("VT_API_KEY")
HEADERS = {
    "accept": "application/json", 
    "x-apikey": API_KEY
    }


Pretendo pasar un bucle con toda la lista de urls. Hay que hacerlo en Base64 encoding

In [52]:
import requests
import base64
import time

def get_url_info(url, retries=2, timeout=10):
    """Consulta la información de la URL en VirusTotal y devuelve resultados relevantes."""
    url_id = base64.urlsafe_b64encode(url.encode()).decode().strip("=")
    api_url = f"https://www.virustotal.com/api/v3/urls/{url_id}"

    for attempt in range(retries + 1):
        try:
            response = requests.get(api_url, headers=HEADERS, timeout=timeout)
            response.raise_for_status()  # Lanza error si status != 200
            data = response.json().get("data", {}).get("attributes", {})

            result = {
                "url": url,
                "last_analysis_stats": data.get("last_analysis_stats"),
                }
            
            return result

        except requests.exceptions.Timeout:
            print(f"Timeout en {url}, intento {attempt+1}/{retries}")
        except requests.exceptions.RequestException as e:
            print(f"Error en {url}: {e}, intento {attempt+1}/{retries}")

        time.sleep(5)  # Pequeña pausa antes de reintentar

    # Si falla después de todos los intentos
    return {"url": url, "error": "No response / failed after retries"}

# Iterar sobre las URLs y obtener resultados
resultados = []
for url in listapd['url']:
    info = get_url_info(url)
    resultados.append(info)
    time.sleep(15)  # Para no superar límites de requests

# Convertir resultados a DataFrame
resultados_df = pd.DataFrame(resultados)

# Mostrar resumen
print(resultados_df.head())

resultados_df.to_json("resultados_vt.json", orient="records", indent=4)



                                     url  \
0  https://conflrma57843.com/u6383644457   
1                https://www.zgahdjm.com   

                                 last_analysis_stats  
0  {'malicious': 18, 'suspicious': 1, 'undetected...  
1  {'malicious': 10, 'suspicious': 0, 'undetected...  


Tipo de respuesta:

url: Contiene la URL que analizada

last_analysis_stats: Es un resumen numérico de los motores de detección de VirusTotal.

Claves típicas:
malicious → número de motores que marcaron la URL como maliciosa.
suspicious → número de motores que consideran la URL sospechosa.
undetected → motores que no detectaron nada malicioso.
harmless → motores que confirmaron que la URL está limpia.


